In [1]:
import tsfresh as ts

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime, timedelta

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pickle

In [2]:
with open('../02_Daten/Kaffeemaschine/preprocessed_coffee_scaled.pickle', 'rb') as handle:
    df = pickle.load(handle)

### DF passend für tsfresh machen

In [3]:
df.head()

,time,day_number,day_name,Status,x,y,z
0,2022-01-28 10:39:28.532,4,Freitag,0,-0.286314,0.185127,-0.094353
1,2022-01-28 10:39:28.742,4,Freitag,0,-0.224631,0.579710,0.564770
2,2022-01-28 10:39:28.915,4,Freitag,0,0.104340,1.368876,0.011101
3,2022-01-28 10:39:29.237,4,Freitag,0,-0.327435,1.095703,-0.278910
4,2022-01-28 10:39:29.455,4,Freitag,0,0.289387,1.186761,-0.331649


In [4]:
df.drop(['day_number','day_name','Status'], axis=1, inplace=True)

In [5]:
df['id'] = 1

In [6]:
df.head()

,time,x,y,z,id
0,2022-01-28 10:39:28.532,-0.286314,0.185127,-0.094353,1
1,2022-01-28 10:39:28.742,-0.224631,0.579710,0.564770,1
2,2022-01-28 10:39:28.915,0.104340,1.368876,0.011101,1
3,2022-01-28 10:39:29.237,-0.327435,1.095703,-0.278910,1
4,2022-01-28 10:39:29.455,0.289387,1.186761,-0.331649,1


In [6]:
df.describe()

,x,y,z,id
count,5.604986e+06,5.604986e+06,5.604986e+06,5604986.0
mean,5.604643e-16,-8.559089e-16,6.900529e-15,1.0
std,1.000000e+00,1.000000e+00,1.000000e+00,0.0
min,-1.004993e+02,-9.466661e+01,-2.179511e+02,1.0
25%,-6.769673e-01,-3.915717e-01,-2.789096e-01,1.0
50%,-1.423885e-01,6.371652e-02,-1.526284e-02,1.0
75%,7.211624e-01,4.582997e-01,2.747600e-01,1.0
max,6.343139e+01,9.518863e+01,1.033632e+02,1.0


## Rolling

mit komplettem Datensatz probieren

In [6]:
df

,time,x,y,z,id
1080000,2022-01-30 15:05:17.526,-0.697528,0.488652,-0.278910,1
1080001,2022-01-30 15:05:17.721,0.248266,0.458300,0.301123,1
1080002,2022-01-30 15:05:17.901,0.063219,-0.270162,-1.412625,1
1080003,2022-01-30 15:05:18.088,-0.676967,0.519005,0.037464,1
1080004,2022-01-30 15:05:18.283,0.145462,-0.330867,0.696600,1
...,...,...,...,...,...
1879995,2022-02-01 06:11:00.297,-0.841453,0.094069,0.195657,1
1879996,2022-02-01 06:11:00.417,-0.347996,-1.089681,-0.621659,1
1879997,2022-02-01 06:11:00.574,-0.347996,-0.270162,-0.727126,1
1879998,2022-02-01 06:11:00.724,-0.368556,-0.270162,0.169294,1


In [7]:
df.time = df.index

c:\Users\pauls\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
df

,time,x,y,z,id
1080000,1080000,-0.697528,0.488652,-0.278910,1
1080001,1080001,0.248266,0.458300,0.301123,1
1080002,1080002,0.063219,-0.270162,-1.412625,1
1080003,1080003,-0.676967,0.519005,0.037464,1
1080004,1080004,0.145462,-0.330867,0.696600,1
...,...,...,...,...,...
1879995,1879995,-0.841453,0.094069,0.195657,1
1879996,1879996,-0.347996,-1.089681,-0.621659,1
1879997,1879997,-0.347996,-0.270162,-0.727126,1
1879998,1879998,-0.368556,-0.270162,0.169294,1


In [9]:
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled = roll_time_series(df, column_id="id", column_sort="time", max_timeshift=250, rolling_direction=4) 
#rolling_direction: The absolute value decides, how much to shift at each step. -> um Zeit und Ressourcen zu sparen, immer 4 Schritte gehen

Rolling: 100%|██████████| 10/10 [1:27:29<00:00, 524.96s/it]


In [10]:
df_rolled

,time,x,y,z,id
0,1080000,-0.697528,0.488652,-0.278910,"(1, 1080003)"
1,1080001,0.248266,0.458300,0.301123,"(1, 1080003)"
2,1080002,0.063219,-0.270162,-1.412625,"(1, 1080003)"
3,1080003,-0.676967,0.519005,0.037464,"(1, 1080003)"
4,1080000,-0.697528,0.488652,-0.278910,"(1, 1080007)"
...,...,...,...,...,...
50192245,1879995,-0.841453,0.094069,0.195657,"(1, 1879999)"
50192246,1879996,-0.347996,-1.089681,-0.621659,"(1, 1879999)"
50192247,1879997,-0.347996,-0.270162,-0.727126,"(1, 1879999)"
50192248,1879998,-0.368556,-0.270162,0.169294,"(1, 1879999)"


In [11]:
# irgendwo speichern, am besten direkt im OneDrive
with open('../Innoprojekt/df_rolled_full.pickle', 'wb') as handle:
    pickle.dump(df_rolled, handle, protocol=pickle.HIGHEST_PROTOCOL)